In [1]:
# ASL Gesture Recognition with 1D CNN
#This notebook demonstrates the process of training a 1D CNN for recognizing American Sign Language gestures using sensor data.

## Step 1: Import Libraries
#```python
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Dropout, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import plot_model


In [2]:
## Step 3: Define Data Directory
# Specify the path to your dataset directory.
# ```python
data_dir = r'C:\Users\Niklesh\Documents\Capstone Project\Project code\my_dataset_robotic_arm'


In [3]:
## Step 4: Feature Extraction Function
# Define a function to extract features from CSV files.
# ```python
def extract_features_from_csv(file_path):
    df = pd.read_csv(file_path)
    features = df.drop(columns=['timestamp', 'user_id']).values
    return features


In [4]:
## Step 5: Load Dataset
# Define a function to load data from the specified directory.
# ```python
def load_data_from_directory(data_dir):
    X, y = [], []
    for user in os.listdir(data_dir):
        user_path = os.path.join(data_dir, user)
        if os.path.isdir(user_path):
            for file in os.listdir(user_path):
                if file.endswith('.csv'):
                    file_path = os.path.join(user_path, file)
                    features = extract_features_from_csv(file_path)
                    gesture_label = os.path.splitext(file)[0]
                    X.append(features)
                    y.extend([gesture_label] * features.shape[0])
    X = np.vstack(X)
    y = np.array(y)
    return X, y


In [5]:
# ## Step 6: Load and Preprocess Data
# Load the dataset and preprocess it.
# ```python
X, y = load_data_from_directory(data_dir)
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for 1D CNN
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)


In [6]:
## Step 7: Convert Labels to Integer Encoding
# Encode the labels as integers.
# ```python
label_set = np.unique(y)
label_map = {label: idx for idx, label in enumerate(label_set)}
y_train_encoded = np.array([label_map[label] for label in y_train])
y_test_encoded = np.array([label_map[label] for label in y_test])


In [ ]:
## Step 8: Define the 1D CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(len(label_set), activation='softmax')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Save the architecture diagram
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)



In [ ]:
# ## Step 9: Train the Model
# Define early stopping and train the model.
# ```python
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train_encoded, epochs=30, batch_size=128,
                    validation_split=0.2, callbacks=[early_stopping], verbose=1)
# Save the trained model
model.save('asl_gesture_model.keras')



In [ ]:
# Make predictions
y_pred_encoded = model.predict(X_test)
y_pred = np.argmax(y_pred_encoded, axis=1)

# Calculate precision, recall, and F1-score for each class
precision, recall, f1, _ = precision_recall_fscore_support(y_test_encoded, y_pred, average=None, zero_division=0)

# Print metrics for each letter
for idx, label in enumerate(label_set):
    print(f"Letter: {label}")
    print(f"  Precision: {precision[idx]:.4f}")
    print(f"  Recall: {recall[idx]:.4f}")
    print(f"  F1 Score: {f1[idx]:.4f}")


In [ ]:
## Step 11: Plot Training History
# Visualize the training and validation accuracy and loss.
# ```python
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate the confusion matrix with unique labels
cm = confusion_matrix(y_test_encoded, y_pred)


# Create the confusion matrix display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_set)

# Plot without displaying values, just colors
fig, ax = plt.subplots(figsize=(10, 6))
disp.plot(ax=ax, include_values=True, colorbar=True)  # Use include_values=False to hide values

plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(y_test_encoded, y_pred)

# Plotting the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=False, fmt="d", cmap="summer", xticklabels=label_set, yticklabels=label_set, cbar=True)

plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
# Save the trained model
#model.save('asl_gesture_model.h5')


In [ ]:
# import tensorflow as tf
# import serial
# import numpy as np
# import time
# import tkinter as tk
# from tkinter import Label

# # Load the saved model
# model = tf.keras.models.load_model('asl_gesture_model.h5')

# # Initialize serial connection (adjust COM port and baud rate as per your device)
# ser = serial.Serial('COM7', 115200, timeout=1)

# # Create a dictionary to map gesture index to label
# #label_set = ['A', 'B', 'C', 'D', 'E']  # Update this with your actual gesture labels

# def get_real_time_data():
#     if ser.in_waiting > 0:
#         try:
#             # Assuming the incoming data is comma-separated for flex sensors and quaternion values
#             data = ser.readline().decode('utf-8').strip().split(',')
#             data = np.array(data, dtype=float)  # Convert to float
#             return data.reshape(1, -1, 1)  # Reshape to fit model input (1, num_features, 1)
#         except:
#             return None
#     return None

# # Function to perform gesture prediction
# def predict_gesture():
#     start_time = time.time()
#     while time.time() - start_time < 15:  # 15 seconds to perform the gesture
#         sensor_data = get_real_time_data()
#         if sensor_data is not None:
#             # Make prediction
#             pred = model.predict(sensor_data)
#             gesture_idx = np.argmax(pred, axis=1)  # Get the index of the highest probability
#             predicted_gesture = label_set[gesture_idx[0]]  # Map to gesture label
#             result_label.config(text=f"Predicted Gesture: {predicted_gesture}")
#             root.update()
#             time.sleep(0.1)  # Small delay to avoid too frequent polling
#     buffer_time()  # Start the buffer after prediction

# # Function to wait 15 seconds as a buffer before next prediction
# def buffer_time():
#     result_label.config(text="Buffer time... Wait for the next gesture.")
#     root.update()
#     time.sleep(15)  # 15 seconds buffer
#     predict_gesture()  # Start prediction again after buffer

# # GUI Setup
# root = tk.Tk()
# root.title("ASL Gesture Recognition")
# root.geometry("400x200")

# # Label to display results
# result_label = Label(root, text="Perform a gesture now...", font=("Helvetica", 16))
# result_label.pack(pady=20)

# # Start prediction immediately when the GUI opens
# predict_gesture()

# root.mainloop()


In [ ]:

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('asl_gesture_model.tflite', 'wb') as f:
    f.write(tflite_model)